# Beta expanded ensemble

Example expanded ensemble simulation inverse temperature.

In [3]:
import feasst as fst

mc = fst.MonteCarlo()
mc.add(fst.MakeConfiguration(fst.args({"cubic_box_length": "8",
                                       "particle_type0": fst.install_dir() + "/forcefield/lj.fstprt"})))
mc.add(fst.MakePotential(fst.MakeLennardJones()))
mc.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1.2", "chemical_potential": "1."})))
beta_min = 0.8
beta_max = 1.2
beta_num = 5
delta_beta = str((beta_max - beta_min)/(beta_num-1))
mc.set(fst.MakeFlatHistogram(
    fst.MakeMacrostateBeta(
        fst.Histogram(fst.args({"width": delta_beta, "max": str(beta_max), "min": str(beta_min)}))),
    fst.MakeWLTM(fst.args({"collect_flatness": "18", "min_flatness": "22", "min_sweeps": "10"}))))
mc.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "1."})))
mc.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
mc.run(fst.MakeRun(fst.args({"until_num_particles": "10"})))
mc.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
mc.add(fst.MakeTrialBeta(fst.args({"fixed_beta_change": delta_beta})))
trials_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"trials_per": trials_per, "file_name": "lj_beta"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"trials_per": trials_per})))
mc.add(fst.MakeCriteriaUpdater(fst.args({"trials_per": trials_per})))
mc.add(fst.MakeCriteriaWriter(fst.args({"trials_per": trials_per, "file_name": "lj_beta_crit.txt",
    "file_name_append_phase": "true"})))
mc.add(fst.MakeEnergy(fst.args({
    "file_name": "lj_beta_energy",
    "file_name_append_phase": "true",
    "trials_per_update": "1",
    "trials_per_write": trials_per,
    "multistate": "true"})))
mc.attempt(int(1e6))

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!